# 第 7 章 数据清洗和准备

In [104]:
import pandas as pd
import numpy as np

## 7.1 处理缺失数据

pandas 的目标之一就是尽量 轻松地处理缺失数据。

对于数值数据，pandas 使用浮点值 NaN(Not a Number)表示缺失数 据。我们称其为哨兵值，可以方便的检测出来:

In [9]:
string_data = pd.Series(['aardvark', 'artichoke', np.nan,'avocado'])

In [10]:
string_data

0     aardvark
1    artichoke
2          NaN
3      avocado
dtype: object

In [11]:
string_data.isnull()

0    False
1    False
2     True
3    False
dtype: bool

Python 内置的 None 值在对象数组中也可以作为 NA:

In [12]:
string_data[0] = None

In [14]:
string_data

0         None
1    artichoke
2          NaN
3      avocado
dtype: object

In [13]:
string_data.isnull()

0     True
1    False
2     True
3    False
dtype: bool

## 滤除缺失数据

你可以通过 pandas.isnull 或布尔索引的手 工方法，但 dropna 可能会更实用一些。对于一个 Series，dropna 返回一个仅 含非空数据和索引值的 Series:

In [15]:
from numpy import nan as NA

In [16]:
data = pd.Series([1, NA, 3.5, NA, 7])

In [19]:
data.dropna() #dropna 返回一个仅 含非空数据和索引值的 Series:

0    1.0
2    3.5
4    7.0
dtype: float64

In [20]:
#这等价于:
data[data.notnull()]

0    1.0
2    3.5
4    7.0
dtype: float64

而对于 DataFrame 对象，事情就有点复杂了。你可能希望丢弃全 NA 或含有 NA 的行或列。dropna 默认丢弃任何含有缺失值的行:

In [41]:
data = pd.DataFrame([[1., 6.5, 3.], [1., NA, NA],
                     [NA, NA, NA], [NA, 6.5, 3.]])

In [23]:
data

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [25]:
cleaned = data.dropna()  #dropna 默认丢弃任何含有缺失值的行
cleaned 

,0,1,2
0,1.0,6.5,3.0


传入 how='all'将只丢弃全为 NA 的那些行:

In [42]:
data.dropna(how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


用这种方式丢弃列，只需传入 axis=1 即可:

In [43]:
data[4] = NA

In [44]:
data

,0,1,2,4
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


In [45]:
data.dropna(axis=1, how='all') #丢弃一列全是NaN的

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


另一个滤除 DataFrame 行的问题涉及时间序列数据。假设你只想留下一部分观 测数据，可以用 thresh 参数实现此目的:

In [49]:
df = pd.DataFrame(np.random.randn(7, 3))
df

,0,1,2
0,-0.828930,0.391449,0.464204
1,0.434492,-0.050273,0.461938
2,-0.440553,0.538571,-1.895166
3,-1.110627,0.782413,-0.792348
4,-1.285666,0.084716,-0.304725
5,-0.037175,1.274717,0.564013
6,1.124009,-0.189275,0.401876


In [51]:
df.iloc[:4, 1] = NA   #直接在原df上修改
df

,0,1,2
0,-0.828930,NaN,0.464204
1,0.434492,NaN,0.461938
2,-0.440553,NaN,-1.895166
3,-1.110627,NaN,-0.792348
4,-1.285666,0.084716,-0.304725
5,-0.037175,1.274717,0.564013
6,1.124009,-0.189275,0.401876


In [52]:
df.iloc[:2, 2] = NA
df

,0,1,2
0,-0.828930,NaN,NaN
1,0.434492,NaN,NaN
2,-0.440553,NaN,-1.895166
3,-1.110627,NaN,-0.792348
4,-1.285666,0.084716,-0.304725
5,-0.037175,1.274717,0.564013
6,1.124009,-0.189275,0.401876


In [53]:
df.dropna() #带有NA的都去掉

,0,1,2
4,-1.285666,0.084716,-0.304725
5,-0.037175,1.274717,0.564013
6,1.124009,-0.189275,0.401876


In [54]:
df.dropna(thresh=2)   #行中带有2个NA的去掉

,0,1,2
2,-0.440553,NaN,-1.895166
3,-1.110627,NaN,-0.792348
4,-1.285666,0.084716,-0.304725
5,-0.037175,1.274717,0.564013
6,1.124009,-0.189275,0.401876


In [75]:
df
df.dropna(thresh=2)   #传入thresh=n保留至少有n个非NaN数据的行：

,0,1,2
2,-0.440553,NaN,-1.895166
3,-1.110627,NaN,-0.792348
4,-1.285666,0.084716,-0.304725
5,-0.037175,1.274717,0.564013
6,1.124009,-0.189275,0.401876


## 填充缺失数据

对于大多数情况而言，fillna 方法是最主要的 函数。通过一个常数调用 fillna 就会将缺失值替换为那个常数值:

In [77]:
df.fillna(0)   #fillna 就会将缺失值替换为常数值

,0,1,2
0,-0.828930,0.000000,0.000000
1,0.434492,0.000000,0.000000
2,-0.440553,0.000000,-1.895166
3,-1.110627,0.000000,-0.792348
4,-1.285666,0.084716,-0.304725
5,-0.037175,1.274717,0.564013
6,1.124009,-0.189275,0.401876


若是通过一个字典调用 fillna，就可以实现对不同的列填充不同的值:

In [78]:
df.fillna({1: 0.5, 2: 0})

,0,1,2
0,-0.828930,0.500000,0.000000
1,0.434492,0.500000,0.000000
2,-0.440553,0.500000,-1.895166
3,-1.110627,0.500000,-0.792348
4,-1.285666,0.084716,-0.304725
5,-0.037175,1.274717,0.564013
6,1.124009,-0.189275,0.401876


fillna 默认会返回新对象，但也可以对现有对象进行就地修改:

In [79]:
_ = df.fillna(0, inplace=True)

In [81]:
df

,0,1,2
0,-0.828930,0.000000,0.000000
1,0.434492,0.000000,0.000000
2,-0.440553,0.000000,-1.895166
3,-1.110627,0.000000,-0.792348
4,-1.285666,0.084716,-0.304725
5,-0.037175,1.274717,0.564013
6,1.124009,-0.189275,0.401876


对 reindexing 有效的那些插值方法也可用于 fillna:

In [83]:
df = pd.DataFrame(np.random.randn(6, 3))
df

,0,1,2
0,0.409496,-0.790201,1.176121
1,-1.660084,0.661740,-0.186250
2,-1.246147,-1.119467,1.163443
3,0.952511,-0.316789,-0.679762
4,1.760643,0.938081,-1.088436
5,-1.953855,-0.214966,-0.341011


In [85]:
df.iloc[2:, 1] = NA
df

,0,1,2
0,0.409496,-0.790201,1.176121
1,-1.660084,0.661740,-0.186250
2,-1.246147,NaN,1.163443
3,0.952511,NaN,-0.679762
4,1.760643,NaN,-1.088436
5,-1.953855,NaN,-0.341011


In [87]:
df.iloc[4:, 2] = NA
df

,0,1,2
0,0.409496,-0.790201,1.176121
1,-1.660084,0.661740,-0.186250
2,-1.246147,NaN,1.163443
3,0.952511,NaN,-0.679762
4,1.760643,NaN,NaN
5,-1.953855,NaN,NaN


In [88]:
df.fillna(method='ffill')  ## todo

,0,1,2
0,0.409496,-0.790201,1.176121
1,-1.660084,0.661740,-0.186250
2,-1.246147,0.661740,1.163443
3,0.952511,0.661740,-0.679762
4,1.760643,0.661740,-0.679762
5,-1.953855,0.661740,-0.679762


In [89]:
df.fillna(method='ffill', limit=2)

,0,1,2
0,0.409496,-0.790201,1.176121
1,-1.660084,0.661740,-0.186250
2,-1.246147,0.661740,1.163443
3,0.952511,0.661740,-0.679762
4,1.760643,NaN,-0.679762
5,-1.953855,NaN,-0.679762


只要有些创新，你就可以利用 fillna 实现许多别的功能。比如说，你可以传入 Series 的平均值或中位数:

In [91]:
data = pd.Series([1., NA, 3.5, NA, 7])
data

0    1.0
1    NaN
2    3.5
3    NaN
4    7.0
dtype: float64

In [92]:
data.fillna(data.mean())

0    1.000000
1    3.833333
2    3.500000
3    3.833333
4    7.000000
dtype: float64

## 7.2 数据转换

## 移除重复数据

DataFrame 中出现重复行有多种原因。下面就是一个例子:

In [93]:
data = pd.DataFrame({'k1': ['one', 'two'] * 3 + ['two'],
                     'k2': [1, 1, 2, 3, 3, 4, 4]})

In [94]:
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


DataFrame 的 duplicated 方法返回一个布尔型 Series，表示各行是否是重复行 (前面出现过的行):

In [95]:
data.duplicated()  #判断各行是否重复

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

还有一个与此相关的 drop_duplicates 方法，它会返回一个 DataFrame，重复 的数组会标为 False:

In [96]:
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


这两个方法默认会判断全部列，你也可以指定部分列进行重复项判断。假设我 们还有一列值，且只希望根据 k1 列过滤重复项:

In [98]:
data['v1'] = range(7)

In [99]:
data

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
5,two,4,5
6,two,4,6


In [100]:
data.drop_duplicates(['k1'])

,k1,k2,v1
0,one,1,0
1,two,1,1


duplicated 和 drop_duplicates 默认保留的是第一个出现的值组合。传入 keep='last'则保留最后一个:

In [102]:
data.drop_duplicates(['k1', 'k2'], keep='last')

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
6,two,4,6


## 利用函数或映射进行数据转换

对于许多数据集，你可能希望根据数组、Series 或 DataFrame 列中的值来实现 转换工作。

In [3]:
data = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon',
                              'Pastrami', 'corned beef', 'Bacon',
                              'pastrami', 'honey ham', 'novalox'],
                     'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,novalox,6.0


In [4]:
meat_to_animal = {
  'bacon': 'pig',
  'pulled pork': 'pig',
  'pastrami': 'cow',
  'corned beef': 'cow',
  'honey ham': 'pig',
  'nova lox': 'salmon'
}

In [5]:
lowercased = data['food'].str.lower()
lowercased

0          bacon
1    pulled pork
2          bacon
3       pastrami
4    corned beef
5          bacon
6       pastrami
7      honey ham
8        novalox
Name: food, dtype: object

In [6]:
data['animal'] = lowercased.map(meat_to_animal)
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,novalox,6.0,NaN


我们也可以传入一个能够完成全部这些工作的函数:

In [8]:
data['food'].map(lambda x: meat_to_animal[x.lower()]) #异常 todo


KeyError: 'novalox'

In [11]:
def meat_2_animal(series):
    if series['food'] == 'bacon':
        return 'pig'
    elif series['food'] == 'pulled pork':
        return 'pig'
    elif series['food'] == 'pastrami':
        return 'cow'
    elif series['food'] == 'corned beef':
        return 'cow'
    elif series['food'] == 'honey ham':
        return 'pig'
    else:
        return 'salmon'

In [12]:
#another way of doing it is: convert the food values to the lower case and apply the function
lower = lambda x: x.lower()
data['food'] = data['food'].apply(lower)
data['animal2'] = data.apply(meat_2_animal, axis='columns')
data

,food,ounces,animal,animal2
0,bacon,4.0,pig,pig
1,pulled pork,3.0,pig,pig
2,bacon,12.0,pig,pig
3,pastrami,6.0,cow,cow
4,corned beef,7.5,cow,cow
5,bacon,8.0,pig,pig
6,pastrami,3.0,cow,cow
7,honey ham,5.0,pig,pig
8,novalox,6.0,NaN,salmon


## 替换值

map 可用于修改对象的数据子集，而 replace 则提供了一种实现该功能的 更简单、更灵活的方式。我们来看看下面这个 Series:


In [19]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

-999 这个值可能是一个表示缺失数据的标记值。要将其替换为 pandas 能够理 解的 NA 值，我们可以利用 replace 来产生一个新的 Series(除非传入 inplace=True):

In [20]:
data.replace(-999, np.nan)


0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

In [18]:
#不需要执行
data.replace(-999, np.nan, inplace=True)     #除非传入 inplace=True,会生成新的data
data

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

In [22]:
#如果你希望一次性替换多个值，可以传入一个由待替换值组成的列表以及一个
替换值::
data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

In [23]:
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

In [24]:
#要让每个值有不同的替换值，可以传递一个替换列表:
data.replace([-999, -1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

In [25]:
#传入的参数也可以是字典:
data.replace({-999: np.nan, -1000: 0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

## 重命名轴索引

In [26]:
#跟 Series 中的值一样，轴标签也可以通过函数或映射进行转换，从而得到一个 新的不同标签的对象。轴还可以被就地修改，而无需新建一个数据结构
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
                    index=['Ohio', 'Colorado', 'New York'],
                    columns=['one', 'two', 'three', 'four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [28]:
transform = lambda x: x[:4].upper()
transform

<function __main__.<lambda>(x)>

In [29]:
data.index.map(transform)

Index(['OHIO', 'COLO', 'NEW '], dtype='object')

In [30]:
#你可以将其赋值给 index，这样就可以对 DataFrame 进行就地修改:
data.index = data.index.map(transform)
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


In [31]:
#如果想要创建数据集的转换版(而不是修改原始数据)，比较实用的方法是 rename: 
data.rename(index=str.title, columns=str.upper)   #str.title 所有单词都是以大写开始

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colo,4,5,6,7
New,8,9,10,11


In [33]:
#rename 可以结合字典型对象实现对部分轴标签的更新:
data.rename(index={'OHIO': 'INDIANA'},
            columns={'three': 'peekaboo'})

,one,two,peekaboo,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


In [34]:
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


In [35]:
#rename 可以实现复制 DataFrame 并对其索引和列标签进行赋值。如果希望就地 修改某个数据集，传入 inplace=True 即可:
data.rename(index={'OHIO': 'INDIANA'}, inplace=True)
data

,one,two,three,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


## 离散化和面元划分

In [36]:
#为了便于分析，连续数据常常被离散化或拆分为“面元”(bin)。假设有一组 人员数据，而你希望将它们划分为不同的年龄组:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]
ages

[20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

In [37]:
bins = [18, 25, 35, 60, 100]
bins

[18, 25, 35, 60, 100]

In [39]:
cats = pd.cut(ages, bins)
cats

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

In [40]:
#pandas 返回的是一个特殊的 Categorical 对象。结果展示了 pandas.cut 划分 的面元。
#你可以将其看做一组表示面元名称的字符串。它的底层含有一个表示 不同分类名称的类型数组，以及一个 codes 属性中的年龄数据的标签:
cats.codes

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [41]:
cats.categories

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]],
              closed='right',
              dtype='interval[int64]')

In [42]:
#pd.value_counts(cats)是 pandas.cut 结果的面元计数。
pd.value_counts(cats)

(18, 25]     5
(35, 60]     3
(25, 35]     3
(60, 100]    1
dtype: int64

In [43]:
#跟“区间”的数学符号一样，圆括号表示开端，而方括号则表示闭端(包 括)。哪边是闭端可以通过 right=False 进行修改:
pd.cut(ages, [18, 26, 36, 61, 100], right=False)

[[18, 26), [18, 26), [18, 26), [26, 36), [18, 26), ..., [26, 36), [61, 100), [36, 61), [36, 61), [26, 36)]
Length: 12
Categories (4, interval[int64]): [[18, 26) < [26, 36) < [36, 61) < [61, 100)]

In [44]:
#你可 以通过传递一个列表或数组到 labels，设置自己的面元名称:
group_names = ['Youth', 'YoungAdult', 'MiddleAged',
'Senior']
group_names

['Youth', 'YoungAdult', 'MiddleAged', 'Senior']

In [45]:
pd.cut(ages, bins, labels=group_names)

[Youth, Youth, Youth, YoungAdult, Youth, ..., YoungAdult, Senior, MiddleAged, MiddleAged, YoungAdult]
Length: 12
Categories (4, object): [Youth < YoungAdult < MiddleAged < Senior]

In [46]:
#如果向 cut 传入的是面元的数量而不是确切的面元边界，则它会根据数据的最 小值和最大值计算等长面元。
#下面这个例子中，我们将一些均匀分布的数据分 成四组:
data = np.random.rand(20)
data

array([0.45182605, 0.73133807, 0.95879353, 0.88132625, 0.08474491,
       0.78829556, 0.90123905, 0.3419702 , 0.52156127, 0.68347542,
       0.47854851, 0.06076258, 0.78309537, 0.05318634, 0.62445089,
       0.33687348, 0.8978933 , 0.89617031, 0.13071648, 0.25516115])

In [47]:
pd.cut(data, 4, precision=2) #选项 precision=2，限定小数只有两位。

[(0.28, 0.51], (0.51, 0.73], (0.73, 0.96], (0.73, 0.96], (0.052, 0.28], ..., (0.28, 0.51], (0.73, 0.96], (0.73, 0.96], (0.052, 0.28], (0.052, 0.28]]
Length: 20
Categories (4, interval[float64]): [(0.052, 0.28] < (0.28, 0.51] < (0.51, 0.73] < (0.73, 0.96]]

In [49]:
#qcut 是一个非常类似于 cut 的函数，它可以根据样本分位数对数据进行面元划 分。
#根据数据的分布情况，cut 可能无法使各个面元中含有相同数量的数据 点。而 qcut 由于使用的是样本分位数，因此可以得到大小基本相等的面元:
data = np.random.randn(1000)  # Normally distributed
cats = pd.qcut(data, 4)  # Cut into quartiles
cats

[(-0.683, -0.00452], (-0.00452, 0.684], (0.684, 3.202], (-3.089, -0.683], (-0.683, -0.00452], ..., (-3.089, -0.683], (-3.089, -0.683], (-0.683, -0.00452], (0.684, 3.202], (0.684, 3.202]]
Length: 1000
Categories (4, interval[float64]): [(-3.089, -0.683] < (-0.683, -0.00452] < (-0.00452, 0.684] < (0.684, 3.202]]

In [50]:
pd.value_counts(cats)

(0.684, 3.202]        250
(-0.00452, 0.684]     250
(-0.683, -0.00452]    250
(-3.089, -0.683]      250
dtype: int64

In [53]:
#与 cut 类似，你也可以传递自定义的分位数(0 到 1 之间的数值，包含端 点):
cats2 = pd.qcut(data, [0, 0.1, 0.5, 0.9, 1.])
cats2

[(-1.281, -0.00452], (-0.00452, 1.363], (-0.00452, 1.363], (-3.089, -1.281], (-1.281, -0.00452], ..., (-3.089, -1.281], (-1.281, -0.00452], (-1.281, -0.00452], (-0.00452, 1.363], (-0.00452, 1.363]]
Length: 1000
Categories (4, interval[float64]): [(-3.089, -1.281] < (-1.281, -0.00452] < (-0.00452, 1.363] < (1.363, 3.202]]

In [54]:
pd.value_counts(cats2)

(-0.00452, 1.363]     400
(-1.281, -0.00452]    400
(1.363, 3.202]        100
(-3.089, -1.281]      100
dtype: int64

备注 ：
numpy.random.randn(d0, d1, …, dn)是从标准正态分布中返回一个或多个样本值。 
numpy.random.rand(d0, d1, …, dn)的随机样本位于[0, 1)中。

## 检测和过滤异常值

In [79]:
#过滤或变换异常值(outlier)在很大程度上就是运用数组运算。来看一个含有 正态分布数据的 DataFrame:
data = pd.DataFrame(np.random.randn(1000, 4))
#对于数值数据，结果的索引将包括计数，平均值，标准差，最小值，最大值以及较低的百分位数和50。
#默认情况下，较低的百分位数为25，较高的百分位数为75.50百分位数与中位数相同。
data.describe()  

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.004685,0.022892,-0.022595,-0.022908
std,1.000359,1.000330,1.006108,0.989050
min,-3.391728,-3.130783,-3.265100,-3.210917
25%,-0.667896,-0.613210,-0.708268,-0.698337
50%,-0.034693,0.017275,0.024936,-0.041598
75%,0.650643,0.691669,0.647747,0.613609
max,3.079191,2.663351,2.684065,2.934604


备注：解释什么是分位数

In [60]:
data = pd.DataFrame(np.random.randn(2, 4))
data

,0,1,2,3
0,0.086708,0.038105,0.288857,-0.909947
1,1.908028,-0.559635,0.490784,-1.020148


In [61]:
data.describe()

,0,1,2,3
count,2.000000,2.000000,2.000000,2.000000
mean,0.997368,-0.260765,0.389820,-0.965047
std,1.287867,0.422666,0.142784,0.077924
min,0.086708,-0.559635,0.288857,-1.020148
25%,0.542038,-0.410200,0.339339,-0.992597
50%,0.997368,-0.260765,0.389820,-0.965047
75%,1.452698,-0.111330,0.440302,-0.937497
max,1.908028,0.038105,0.490784,-0.909947


In [62]:
data[0]

0    0.086708
1    1.908028
Name: 0, dtype: float64

In [64]:
percent_data = np.percentile(data[0],25) #25%分位数
percent_data

0.5420382555534708

In [66]:
median_data = np.median(data[0])  #中位数
median_data

0.9973681311002018

In [80]:
#假设你想要找出某列中绝对值大小超过 3 的值:
col = data[2]
col

0      0.837567
1      1.316184
2      0.790814
3      1.109718
4     -1.482562
5     -1.133333
6      0.153480
7     -0.244604
8     -0.854865
9      1.318079
10    -0.029659
11    -0.266428
12     0.126907
13     1.496480
14    -1.206022
15    -0.789179
16     1.684080
17     2.406043
18    -1.478214
19     0.742774
20     0.419817
21     2.265368
22     0.849194
23     0.870309
24    -0.788987
25    -0.756314
26    -0.036714
27     1.180400
28     1.022249
29     0.893146
         ...   
970    0.996877
971   -0.118179
972    1.489183
973    0.597933
974    0.730714
975   -1.293752
976   -1.684639
977    0.263202
978    1.516205
979    0.386479
980    1.624292
981   -0.704399
982   -0.968919
983   -0.936292
984    1.851223
985   -0.313147
986   -0.911738
987    0.054465
988   -0.597054
989    1.151823
990    2.009181
991   -0.395637
992    0.584092
993    0.353703
994    0.041118
995   -0.918872
996    0.808416
997    0.282393
998   -0.406938
999   -0.040779
Name: 2, Length: 1000, d

In [81]:
col[np.abs(col) > 3]  #绝对值大小abs

192   -3.265100
334   -3.125378
Name: 2, dtype: float64

In [82]:
#要选出全部含有“超过 3 或-3 的值”的行，你可以在布尔型 DataFrame 中使 用 any 方法:
data[(np.abs(data) > 3).any(1)]

,0,1,2,3
192,0.739267,-1.884524,-3.265100,-0.000176
334,1.746179,-0.470427,-3.125378,0.583966
520,1.453667,-3.075868,-0.780860,1.604039
591,-3.391728,0.211336,1.109806,-0.194012
719,-1.336901,-0.554261,0.811331,-3.210917
840,-1.922956,-3.130783,-0.436341,0.600679
930,3.079191,-1.093604,-1.775716,-0.213145


In [83]:
#下面的代码可以将值限制在区间-3 到 3 以内:   （未看懂todo）
data[np.abs(data) > 3] = np.sign(data) * 3
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.004998,0.023099,-0.022204,-0.022697
std,0.998863,0.999693,1.004908,0.988391
min,-3.000000,-3.000000,-3.000000,-3.000000
25%,-0.667896,-0.613210,-0.708268,-0.698337
50%,-0.034693,0.017275,0.024936,-0.041598
75%,0.650643,0.691669,0.647747,0.613609
max,3.000000,2.663351,2.684065,2.934604


In [85]:
#根据数据的值是正还是负，np.sign(data)可以生成 1 和-1:
np.sign(data).head()

,0,1,2,3
0,1.0,-1.0,1.0,-1.0
1,-1.0,1.0,1.0,1.0
2,1.0,1.0,1.0,1.0
3,-1.0,1.0,1.0,1.0
4,1.0,-1.0,-1.0,1.0


## 排列和随机采样

In [ ]:
#利用 numpy.random.permutation 函数可以轻松实现对 Series 或 DataFrame 的 列的排列工作(permuting，随机重排序)。
#通过需要排列的轴的长度调用 permutation，可产生一个表示新顺序的整数数组:

In [87]:
df = pd.DataFrame(np.arange(5 * 4).reshape((5, 4)))
df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [89]:
sampler = np.random.permutation(5)
sampler

array([0, 1, 4, 3, 2])

In [90]:
#然后就可以在基于 iloc 的索引操作或 take 函数中使用该数组了:
df.take(sampler)

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
4,16,17,18,19
3,12,13,14,15
2,8,9,10,11


In [91]:
#如果不想用替换的方式选取随机子集，可以在 Series 和 DataFrame 上使用 sample 方法:
df.sample(n=3)

,0,1,2,3
4,16,17,18,19
1,4,5,6,7
2,8,9,10,11


In [92]:
#要通过替换的方式产生样本(允许重复选择)，可以传递 replace=True 到 sample:
choices = pd.Series([5, 7, -1, 6, 4])
choices

0    5
1    7
2   -1
3    6
4    4
dtype: int64

In [93]:
draws = choices.sample(n=10, replace=True)    #replace不能设置成false
draws

1    7
1    7
3    6
1    7
4    4
2   -1
0    5
3    6
0    5
2   -1
dtype: int64

## 计算指标/哑变量

In [ ]:
#另一种常用于统计建模或机器学习的转换方式是:将分类变量(categorical variable)转换为“哑变量”或“指标矩阵”。

In [98]:
#如果 DataFrame 的某一列中含有 k 个不同的值，则可以派生出一个 k 列矩阵或 DataFrame(其值全为 1 和 0)。
#pandas 有一个 get_dummies 函数可以实现该功 能(其实自己动手做一个也不难)。
df = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                   'data1': range(6)})
df

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [100]:
df['key']

0    b
1    b
2    a
3    c
4    a
5    b
Name: key, dtype: object

In [99]:
pd.get_dummies(df['key'])

,a,b,c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In [101]:
#你可能想给指标 DataFrame 的列加上一个前缀，以便能够跟其他数据 进行合并。get_dummies 的 prefix 参数可以实现该功能:
dummies = pd.get_dummies(df['key'], prefix='key')
dummies

,key_a,key_b,key_c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In [102]:
df_with_dummy = df[['data1']].join(dummies)
df_with_dummy

,data1,key_a,key_b,key_c
0,0,0,1,0
1,1,0,1,0
2,2,1,0,0
3,3,0,0,1
4,4,1,0,0
5,5,0,1,0


In [108]:
#如果 DataFrame 中的某行同属于多个分类，则事情就会有点复杂。看一下 MovieLens 1M 数据集
mnames = ['movie_id', 'title', 'genres']

In [109]:
movies = pd.read_table('datasets/movielens/movies.dat',sep='::',
                       header=None, names=mnames)

/Users/xl/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: read_table is deprecated, use read_csv instead.
  
/Users/xl/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


In [110]:
movies

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [111]:
movies[:10]

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [112]:
#要为每个 genre 添加指标变量就需要做一些数据规整操作。首先，我们从数据 集中抽取出不同的 genre 值:
all_genres = []
for x in movies.genres:
    all_genres.extend(x.split('|'))

In [113]:
genres = pd.unique(all_genres)

In [114]:
genres

array(['Animation', "Children's", 'Comedy', 'Adventure', 'Fantasy',
       'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror',
       'Sci-Fi', 'Documentary', 'War', 'Musical', 'Mystery', 'Film-Noir',
       'Western'], dtype=object)

In [115]:
#构建指标 DataFrame 的方法之一是从一个全零 DataFrame 开始:

In [116]:
zero_matrix = np.zeros((len(movies), len(genres)))
zero_matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [117]:
dummies = pd.DataFrame(zero_matrix, columns=genres)
dummies

,Animation,Children's,Comedy,Adventure,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Sci-Fi,Documentary,War,Musical,Mystery,Film-Noir,Western
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [119]:
#迭代每一部电影，并将 dummies 各行的条目设为 1。要这么做，我们使 用 dummies.columns 来计算每个类型的列索引:
gen = movies.genres[0]
gen

"Animation|Children's|Comedy"

In [120]:
gen.split('|')

['Animation', "Children's", 'Comedy']

In [121]:
dummies.columns.get_indexer(gen.split('|'))

array([0, 1, 2])

In [122]:
#根据索引，使用.iloc 设定值:
for i, gen in enumerate(movies.genres):
    indices = dummies.columns.get_indexer(gen.split('|'))
    dummies.iloc[i, indices] = 1

In [123]:
#和以前一样，再将其与 movies 合并起来:
movies_windic = movies.join(dummies.add_prefix('Genre_'))
movies_windic

,movie_id,title,genres,Genre_Animation,Genre_Children's,Genre_Comedy,Genre_Adventure,Genre_Fantasy,Genre_Romance,Genre_Drama,...,Genre_Crime,Genre_Thriller,Genre_Horror,Genre_Sci-Fi,Genre_Documentary,Genre_War,Genre_Musical,Genre_Mystery,Genre_Film-Noir,Genre_Western
0,1,Toy Story (1995),Animation|Children's|Comedy,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji (1995),Adventure|Children's|Fantasy,0.0,1.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men (1995),Comedy|Romance,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale (1995),Comedy|Drama,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II (1995),Comedy,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,6,Heat (1995),Action|Crime|Thriller,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,7,Sabrina (1995),Comedy|Romance,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,8,Tom and Huck (1995),Adventure|Children's,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,9,Sudden Death (1995),Action,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,10,GoldenEye (1995),Action|Adventure|Thriller,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [124]:
movies_windic.iloc[0]

movie_id                                       1
title                           Toy Story (1995)
genres               Animation|Children's|Comedy
Genre_Animation                                1
Genre_Children's                               1
Genre_Comedy                                   1
Genre_Adventure                                0
Genre_Fantasy                                  0
Genre_Romance                                  0
Genre_Drama                                    0
Genre_Action                                   0
Genre_Crime                                    0
Genre_Thriller                                 0
Genre_Horror                                   0
Genre_Sci-Fi                                   0
Genre_Documentary                              0
Genre_War                                      0
Genre_Musical                                  0
Genre_Mystery                                  0
Genre_Film-Noir                                0
Genre_Western       

In [125]:
#笔记:对于很大的数据，用这种方式构建多成员指标变量就会变得非常慢。
#最好使用更低级的函数，将其写入 NumPy 数组，然后结果包装在 DataFrame 中。

In [127]:
#一个对统计应用有用的秘诀是:结合 get_dummies 和诸如 cut 之类的离散化函 数:
np.random.seed(12345) #原来每次运行代码时设置相同的seed，则每次生成的随机数也相同，如果不设置seed，则每次生成的随机数都会不一样。

In [128]:
values = np.random.rand(10)
values

array([0.92961609, 0.31637555, 0.18391881, 0.20456028, 0.56772503,
       0.5955447 , 0.96451452, 0.6531771 , 0.74890664, 0.65356987])

In [129]:
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]

In [130]:
pd.get_dummies(pd.cut(values, bins))

   (0.0, 0.2]  (0.2, 0.4]  (0.4, 0.6]  (0.6, 0.8]  (0.8, 1.0]
0           0           0           0           0           1
1           0           1           0           0           0
2           1           0           0           0           0
3           0           1           0           0           0
4           0           0           1           0           0
5           0           0           1           0           0
6           0           0           0           0           1
7           0           0           0           1           0
8           0           0           0           1           0
9           0           0           0           1           0

In [ ]:
#我们用 numpy.random.seed，使这个例子具有确定性。

## 7.3 字符串操作

## 字符串对象方法

In [132]:
#以逗号分隔的字符串可以用 split 拆分成数段:
val = 'a,b,  guido'
val.split(',')

['a', 'b', '  guido']

In [133]:
#split 常常与 strip 一起使用，以去除空白符(包括换行符):
pieces = [x.strip() for x in val.split(',')]
pieces

['a', 'b', 'guido']

In [134]:
#利用加法，可以将这些子字符串以双冒号分隔符的形式连接起来:
first, second, third = pieces
first + '::' + second + '::' + third

'a::b::guido'

In [135]:
#另一种方式：但这种方式并不是很实用。一种更快更符合 Python 风格的方式是，向字符串 "::"的 join 方法传入一个列表或元组:
'::'.join(pieces)

'a::b::guido'

In [136]:
#子串定位。检测子串的最佳方式是利用 Python 的 in 关键 字，还可以使用 index 和 find:
'guido' in val

True

In [138]:
val.index(',')

1

In [139]:
val.find(':')

-1

In [140]:
#注意 find 和 index 的区别:如果找不到字符串，index 将会引发一个异常(而 不是返回-1):
val.index('：')

ValueError: substring not found

In [141]:
#count 可以返回指定子串的出现次数:
val.count(',')

2

In [142]:
#replace 用于将指定模式替换为另一个模式。通过传入空字符串，它也常常用 于删除模式:
val.replace(',', '::')

'a::b::  guido'

In [143]:
val.replace(',', '')

'ab  guido'

## 正则表达式

In [ ]:
#正则表达式，常称作 regex，是根据正则表达式语言编写的字符串。Python 内置的 re 模块负责对字符串应用正则表达式。

In [ ]:
#re 模块的函数可以分为三个大类:模式匹配、替换以及拆分。 

In [144]:
#假设我想要拆分一个字符串，分隔符为数量不定的一组空白符(制表符、空格、换行符等)。描述一个或多个空白 符的 regex 是\s+:
import re

In [145]:
text = "foo    bar\t baz  \tqux"

In [146]:
re.split('\s+', text)

['foo', 'bar', 'baz', 'qux']

In [ ]:
#调用 re.split('\s+',text)时，正则表达式会先被编译，然后再在 text 上调 用其 split 方法。
#你可以用 re.compile 自己编译 regex 以得到一个可重用的 regex 对象

In [147]:
regex = re.compile('\s+')

In [148]:
regex.split(text)

['foo', 'bar', 'baz', 'qux']

In [149]:
#如果只希望得到匹配 regex 的所有模式，则可以使用 findall 方法:
regex.findall(text)

['    ', '\t ', '  \t']

In [ ]:
#match 和 search 跟 findall 功能类似。
#findall 返回的是字符串中所有的匹配项，而 search 则只返回第一个匹配项。match 更加严格，它只匹配字符串的首 部。

In [150]:
text = """Dave dave@google.com
Steve steve@gmail.com
Rob rob@gmail.com
Ryan ryan@yahoo.com
"""

In [151]:
pattern = r'[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}'

In [152]:
# re.IGNORECASE makes the regex case-insensitive
regex = re.compile(pattern, flags=re.IGNORECASE)

In [153]:
#对 text 使用 findall 将得到一组电子邮件地址:
regex.findall(text)

['dave@google.com', 'steve@gmail.com', 'rob@gmail.com', 'ryan@yahoo.com']

In [154]:
#search 返回的是文本中第一个电子邮件地址(以特殊的匹配项对象形式返 回)。
#对于上面那个 regex，匹配项对象只能告诉我们模式在原字符串中的起 始和结束位置:
m = regex.search(text)
m

<re.Match object; span=(5, 20), match='dave@google.com'>

In [155]:
text[m.start():m.end()]

'dave@google.com'

In [156]:
#regex.match 则将返回 None，因为它只匹配出现在字符串开头的模式:   （不懂todo）
print(regex.match(text))

None


In [157]:
#sub 方法可以将匹配到的模式替换为指定字符串，并返回所得到的新 字符串:
print(regex.sub('REDACTED', text))

Dave REDACTED
Steve REDACTED
Rob REDACTED
Ryan REDACTED



In [158]:
#假设你不仅想要找出电子邮件地址，还想将各个地址分成 3 个部分:用户名、 域名以及域后缀。
#要实现此功能，只需将待分段的模式的各部分用圆括号包起 来即可:
pattern = r'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})'

In [159]:
regex = re.compile(pattern, flags=re.IGNORECASE)

In [ ]:
#由这种修改过的正则表达式所产生的匹配项对象，可以通过其 groups 方法返回 一个由模式各段组成的元组:

In [160]:
m = regex.match('wesm@bright.net')

In [161]:
m.groups()

('wesm', 'bright', 'net')

In [162]:
#对于带有分组功能的模式，findall 会返回一个元组列表:
regex.findall(text)

[('dave', 'google', 'com'),
 ('steve', 'gmail', 'com'),
 ('rob', 'gmail', 'com'),
 ('ryan', 'yahoo', 'com')]

In [163]:
#sub 还能通过诸如\1、\2 之类的特殊符号访问各匹配项中的分组。符号\1 对应 第一个匹配的组，\2 对应第二个匹配的组，以此类推:
print(regex.sub(r'Username: \1, Domain: \2, Suffix: \3',
text))

Dave Username: dave, Domain: google, Suffix: com
Steve Username: steve, Domain: gmail, Suffix: com
Rob Username: rob, Domain: gmail, Suffix: com
Ryan Username: ryan, Domain: yahoo, Suffix: com



## pandas 的矢量化字符串函数

In [164]:
#清理待分析的散乱数据时，常常需要做一些字符串规整化工作。更为复杂的情况是，含有字符串的列有时还含有缺失数据:
data = {'Dave': 'dave@google.com', 'Steve':
'steve@gmail.com',
        'Rob': 'rob@gmail.com', 'Wes': np.nan}
data

{'Dave': 'dave@google.com',
 'Steve': 'steve@gmail.com',
 'Rob': 'rob@gmail.com',
 'Wes': nan}

In [165]:
data = pd.Series(data)
data

Dave     dave@google.com
Steve    steve@gmail.com
Rob        rob@gmail.com
Wes                  NaN
dtype: object

In [166]:
data.isnull()

Dave     False
Steve    False
Rob      False
Wes       True
dtype: bool

In [ ]:
#通过 data.map，所有字符串和正则表达式方法都能被应用于(传入 lambda 表 达式或其他函数)各个值，但是如果存在 NA(null)就会报错。
#为了解决这个 问题，Series 有一些能够跳过 NA 值的面向数组方法，进行字符串操作。通过 Series 的 str 属性即可访问这些方法。

In [167]:
#我们可以通过 str.contains 检 查各个电子邮件地址是否含有"gmail":
data.str.contains('gmail')

Dave     False
Steve     True
Rob       True
Wes        NaN
dtype: object

In [168]:
#也可以使用正则表达式，还可以加上任意 re 选项(如 IGNORECASE):
pattern

'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\\.([A-Z]{2,4})'

In [169]:
data.str.findall(pattern, flags=re.IGNORECASE)

Dave     [(dave, google, com)]
Steve    [(steve, gmail, com)]
Rob        [(rob, gmail, com)]
Wes                        NaN
dtype: object

In [170]:
#有两个办法可以实现矢量化的元素获取操作:要么使用 str.get，要么在 str 属性上使用索引:
matches = data.str.match(pattern, flags=re.IGNORECASE)
matches

Dave     True
Steve    True
Rob      True
Wes       NaN
dtype: object

In [171]:
#要访问嵌入列表中的元素，我们可以传递索引到这两个函数中:    （没看懂todo）
matches.str.get(1)

Dave    NaN
Steve   NaN
Rob     NaN
Wes     NaN
dtype: float64

In [172]:
matches.str[0]

Dave    NaN
Steve   NaN
Rob     NaN
Wes     NaN
dtype: float64

In [180]:
#你可以利用这种方法对字符串进行截取:
data.str[:5]

Dave     dave@
Steve    steve
Rob      rob@g
Wes        NaN
dtype: object

## 7.4 总结